In [1]:
pip install numpy==1.23.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
bigframes 1.40.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.23.5 w

In [2]:
!pip install gym torch numpy


   ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/363.4 MB 166.0 MB/s eta 0:00:02
ERROR: Operation cancelled by user
^C


In [3]:
!pip install --upgrade gym


^C


In [ ]:
import numpy as np
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# Policy Network Definition
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, action_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, state):
        # # First hidden layer with ReLU
        x = torch.relu(self.fc1(state))
        # Output probabilities of actions
        x = self.softmax(self.fc2(x))
        return x


class REINFORCEAgent:
    def __init__(self, state_dim, action_dim, lr=0.01, gamma=0.99):
        self.policy = PolicyNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma
        self.episode_rewards = []
        self.episode_log_probs = []

    def select_action(self, state):
        state = torch.tensor(state, dtype=torch.float32)
        action_probs = self.policy(state)
        distribution = Categorical(action_probs)
        action = distribution.sample()
        self.episode_log_probs.append(distribution.log_prob(action))
        return action.item()

    def store_reward(self, reward):
        self.episode_rewards.append(reward)

    def update_policy(self):
        G = 0
        returns = []
        for r in reversed(self.episode_rewards):
            G = r + self.gamma * G
            returns.insert(0, G)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        loss = []
        for log_prob, G_t in zip(self.episode_log_probs, returns):
            loss.append(-log_prob * G_t)

        self.optimizer.zero_grad()
        loss = torch.stack(loss).sum()
        loss.backward()
        self.optimizer.step()

        self.episode_rewards = []
        self.episode_log_probs = []

# 训练 REINFORCE Agent
def train_reinforce(num_episodes=1000):
    env = gym.make("CartPole-v1")  # 直接创建环境，无需手动注册
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = REINFORCEAgent(state_dim, action_dim)

    for episode in range(num_episodes):
        state, _ = env.reset()  # 兼容新 Gym API
        done = False
        total_reward = 0

        while not done:
            action = agent.select_action(state)
            next_state, reward, done, truncated, _ = env.step(action)  # 兼容新 Gym API
            agent.store_reward(reward)
            state = next_state
            total_reward += reward

        agent.update_policy()

        if episode % 100 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

    env.close()
    print("training cpmplete！")

if __name__ == "__main__":
    train_reinforce()


Episode 0, Total Reward: 17.0


KeyboardInterrupt: 